# Part 4: Agentic RAG with a Custom MCP Server

**[EN]** This notebook generates the necessary configuration to connect Claude Desktop to your remote MCP server.
**[KR]** 이 노트북은 원격 MCP 서버에 Claude Desktop을 연결하기 위한 설정을 생성합니다.

## Step 1: Prerequisites

**[EN]** Before running this notebook, ensure you have started your server on the remote machine (e.g., EC2) using the `docker run` command from the `README.md`. You will also need the official `mcp-proxy` installed locally (`pip install mcp-proxy`).<br>
**[KR]** 이 노트북을 실행하기 전에, 원격 머신(예: EC2)에서 `README.md`에 안내된 `docker run` 명령어를 사용하여 서버를 시작했는지 확인해주세요. 또한 로컬에 공식 `mcp-proxy`가 설치되어 있어야 합니다 (`pip install mcp-proxy`).

## Step 2: Generate Configuration

**[EN]** Run the cell below. It will ask for your remote server's public IP address and generate the correct JSON configuration for Claude Desktop.<br>
**[KR]** 아래 셀을 실행하세요. 원격 서버의 공인 IP 주소를 입력받아 올바른 Claude Desktop용 JSON 설정을 생성합니다.

In [ ]:
import json

REMOTE_SERVER_IP = input("Please enter the public IP address of your remote server: ")

config = {
    "mcpServers": {
        "colpali-search-server": {
            "command": "mcp-proxy",
            "args": [
                "--transport=streamablehttp",
                f"http://{REMOTE_SERVER_IP}:8000/mcp"
            ]
        }
    }
}

print("\n✅ Configuration generated successfully!")
print("👇 Copy the JSON below and paste it into your local claude_desktop_config.json file:")
print("-" * 70)
print(json.dumps(config, indent=4))
print("-" * 70)

## Step 3: Final Steps

**[EN]** 
1. **Open your Claude Desktop configuration file.**
2. **Paste the configuration.**
3. **Restart Claude Desktop.**

**[KR]** 
1. **Claude Desktop 설정 파일을 엽니다.**
2. **설정을 붙여넣습니다.**
3. **Claude Desktop을 재시작하세요.**